In [12]:
# to upload the request

In [20]:
import os
import tempfile
def upload_file(input_location,sm_session,prefix):

    return sm_session.upload_data(
        input_location,
        bucket=sm_session.default_bucket(),
        key_prefix=prefix,
        extra_args={"ContentType": "application/json"},
    )
def upload_request(request_dict,sm_session,bucket_prefix):
    '''
    dummy_request_location = "async_request.json"
    with open(dummy_request_location,'w') as f:
        json.dump(request_dict,f)    
    os.system(f'rm {dummy_request_location}')        
    '''
    with tempfile.NamedTemporaryFile(mode='w+',suffix='.json',delete=True) as f:
        json.dump(request_dict,f)    
        f.seek(0)
        input_location = f.name
        request_s3_location = upload_file(input_location,sm_session,bucket_prefix)

    return request_s3_location

In [21]:
# to download the response

In [22]:
import urllib, time
from botocore.exceptions import ClientError


def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sm_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("waiting for output...")
                time.sleep(2)
                continue
            raise



In [31]:
%%time
import sagemaker
import boto3
import base64
import json
import botocore.client
endpoint = 'dummy-whisper-async-endpoint'
 
runtime = boto3.Session().client('sagemaker-runtime', 
                                 config=botocore.config.Config(retries={'max_attempts': 0},
                                connect_timeout=900,
                                read_timeout=900)
                                )
 


sm_session = sagemaker.session.Session()
# sm_runtime = boto_session.client("sagemaker-runtime")
# https://chrissardegna.com/blog/why-my-lambdas-timeout/
# https://aws.amazon.com/about-aws/whats-new/2021/08/amazon-sagemaker-asynchronous-new-inference-option/
# Read image into memory
# with open(image, 'rb') as f:
#     payload = f.read()
#=====================================================================
# audio_path = '/home/ec2-user/SageMaker/Audio_James_Randi_Long_Version.ogg'
# audio_path = '/home/ec2-user/SageMaker/Syllable_timing_isochrony.ogg'
# audio_path = '/home/ec2-user/SageMaker/Audio_James_Randi.wav'
audio_path = '/home/ec2-user/SageMaker/thecantervilleghostversion2_01_wilde_128kb.mp3'
#=====================================================================
audio_base64 = base64.b64encode(open(audio_path, "rb").read())
ext = audio_path.split('.')[-1]

payload = audio_base64
# bucket_prefix = "s3://sagemaker-ap-northeast-1-268945887964/dummy-whisper-async/request"
bucket_prefix = "dummy-whisper-async/request"
request_s3_location = upload_request(
                    {
                        'data':payload.decode(),
#                         'data':'hi',
                       'ext':ext},
                       sm_session,
                       bucket_prefix)
print('request_s3_location',request_s3_location)
# assert False
print(f"Invoking Endpoint with async")
print(request_s3_location)
response = runtime.invoke_endpoint_async(
        EndpointName=endpoint, 
        InputLocation=request_s3_location,
    )
output_location = response["OutputLocation"]
result = get_output(output_location)
result = eval(result)
print(result['text'])
'''
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(
    EndpointName=endpoint, 
    ContentType='application/json', 
    Body=json.dumps({'data':payload.decode(),
                     'ext':ext})
)
# Unpack response
result = json.loads(response['Body'].read().decode())
print(result['text'])
'''


request_s3_location s3://sagemaker-ap-northeast-1-268945887964/dummy-whisper-async/request/tmpt0yqr907.json
Invoking Endpoint with async
s3://sagemaker-ap-northeast-1-268945887964/dummy-whisper-async/request/tmpt0yqr907.json
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiti

"\n# Send image via InvokeEndpoint API\nresponse = runtime.invoke_endpoint(\n    EndpointName=endpoint, \n    ContentType='application/json', \n    Body=json.dumps({'data':payload.decode(),\n                     'ext':ext})\n)\n# Unpack response\nresult = json.loads(response['Body'].read().decode())\nprint(result['text'])\n"

## files
1. thecantervilleghostversion2_01_wilde_128kb.mp3 - output in 1min 52s
2. Audio_James_Randi.wav - output in 28.4 s
3. Syllable_timing_isochrony.ogg - 30 s
4. Audio_James_Randi_Long_Version.ogg - 46.4 s 
